### Question 2: What languages were the most popular in 2022? 

We will look at the result survey to the question of what languages the developers have worked with. 

In [1]:
import pandas as pd
from collections import defaultdict
import plotly.express as px
df = pd.read_csv('./survey_results_public_2022.csv')
df.head()

,ResponseId,MainBranch,Employment,RemoteWork,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,LearnCodeCoursesCert,YearsCode,...,TimeSearching,TimeAnswering,Onboarding,ProfessionalTech,TrueFalse_1,TrueFalse_2,TrueFalse_3,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,None of these,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,"Employed, full-time",Fully remote,Hobby;Contribute to open-source projects,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Difficult,NaN
2,3,"I am not primarily a developer, but I write co...","Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",Books / Physical media;Friend or family member...,Technical documentation;Blogs;Programming Game...,NaN,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Neither easy nor difficult,40205.0
3,4,I am a developer by profession,"Employed, full-time",Fully remote,I don’t code outside of work,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Books / Physical media;School (i.e., Universit...",NaN,NaN,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Appropriate in length,Easy,215232.0
4,5,I am a developer by profession,"Employed, full-time","Hybrid (some remote, some in-person)",Hobby,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)","Other online resources (e.g., videos, blogs, f...",Technical documentation;Blogs;Stack Overflow;O...,NaN,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Too long,Easy,NaN


In [3]:
# We can see that the column 'LanguageHaveWorkedWith' shows a combination of languages i.e many languages are included in a response. 
# We will need to find a way to count the occurences of each language seperately, instead of each combination response like that. 
# Create another dataframe by slicing the 'LanguageHaveWorkedWith' column from the original dataframe, and use the value_counts method
# to count the occurences of each combination. Call this df worked_languages. 
worked_languages = df[['LanguageHaveWorkedWith']].value_counts().reset_index()
worked_languages.rename(columns={'LanguageHaveWorkedWith':'Language',0:'count'},inplace=True)
worked_languages.head()

,Language,count
0,HTML/CSS;JavaScript;TypeScript,1250
1,Python,962
2,HTML/CSS;JavaScript,914
3,HTML/CSS;JavaScript;PHP;SQL,745
4,C#;HTML/CSS;JavaScript;SQL;TypeScript,570


In [4]:
# create a list of unique languages so that we can count the occurrences for each language 
worked_languages_df = worked_languages['Language'].str.split(';', expand=True)
no_worked_languages_list = [*range(0,worked_languages_df.shape[1])]
language_list=[]
for i in no_worked_languages_list:
    language_list += worked_languages_df[i].unique().tolist()

In [5]:
# create a function to count the occurences of each language
def count(df,col1,col2,look_for):
    ''' 
    INPUT:
    df - the sliced dataframe as the result of value_counts method 
    col1 - the column with the combinations of values
    col2 - the column with the count value for the combination
    look_for - the list of unique values 

    OUTPUT:
    new_df - a new dataframe with 2 columns, 1 column is the unique value, 1 column is the count of that unique value
    '''
    new_df = defaultdict(int)
    for val in look_for:
        for idx in range(df.shape[0]):
            if (val is not None) and (val in df[col1][idx]):
                new_df[val] += int(df[col2][idx])   
    new_df = pd.DataFrame(pd.Series(new_df)).reset_index()
    new_df.columns = [col1, col2]
    new_df.sort_values('count', ascending=False, inplace=True)
    return new_df

In [6]:
# Now we call the count function on the worked_languages df.
worked_languages_count=count(worked_languages,'Language','count',language_list)
worked_languages_count

,Language,count
15,SQL,1334826
3,Java,1071060
1,Python,1024650
4,JavaScript,975303
19,TypeScript,965328
0,HTML/CSS,743698
11,PHP,429983
20,R,402876
30,PowerShell,248675
17,Rust,225250


In [7]:
# We will filter only the top 10 languages, and create a horizontal bar chart to show the popularity of these top 10 languages 
top_10 =worked_languages_count.head(10)
top_10 =top_10.sort_values("count",ascending=False)
top_10 

,Language,count
15,SQL,1334826
3,Java,1071060
1,Python,1024650
4,JavaScript,975303
19,TypeScript,965328
0,HTML/CSS,743698
11,PHP,429983
20,R,402876
30,PowerShell,248675
17,Rust,225250


In [8]:
# Using plotly to create a horizontal bar chart to show the popularity of these top 10 languages. 
fig = px.bar(top_10, x="count", y="Language", orientation='h',title="Top 10 Languages Developers Worked with in 2022",text="count")
fig.show()

### Refer to Chart 2.png for visualization